<a href="https://colab.research.google.com/github/swKyungbock/2023MLwithTextData/blob/main/6%EC%9E%A5_%EA%B5%AD%EB%AF%BC%EC%B2%AD%EC%9B%90%EB%8D%B0%EC%9D%B4%ED%84%B0_%EC%9D%B4%EC%A7%84%EB%B6%84%EB%A5%98(LightGBM).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 국민청원 데이터로 이진 분류하기

In [ ]:
import os
import re
import platform
import pandas as pd
import numpy as np

# 데이터 로드하기
*  데이터 출처 : https://github.com/akngs/petitions

In [ ]:
base_path = "data"
file_name = "petition.csv"

def file_exist_check(base_path, file_name):
    if os.path.exists(f"./{base_path}/{file_name}"):
        print(f"{os.getcwd()}/{base_path} 경로에 파일이 있음")
        return

    if not os.path.exists(base_path):
        os.makedirs(base_path)

    if platform.system() == "Linux":
        # 구글 드라이브에서 csv 파일을 읽어오기 위해 gauth 인증하기
        !pip install -U -q PyDrive
        from pydrive.auth import GoogleAuth
        from pydrive.drive import GoogleDrive
        from google.colab import auth
        from oauth2client.client import GoogleCredentials

        auth.authenticate_user()
        gauth = GoogleAuth()
        gauth.credentials = GoogleCredentials.get_application_default()
        drive = GoogleDrive(gauth)
        url ='https://drive.google.com/open?id=1d8VEAj6n83wT1YRlCrhoU_1mMSvYpXc4'

        id = url.split('=')[1]
        downloaded = drive.CreateFile({'id':id})
        downloaded.GetContentFile(f'{base_path}/{file_name}')
        print(f"사용자의 구글 드라이브에 {base_path}/{file_name} 다운로드 완료")
        return
    else:
        print(f"{file_link} 에서 다운로드 받아 실습 경로 {os.getcwd()}/{base_path}에 옮겨주세요.""")
        return

file_exist_check(base_path, file_name)

In [ ]:
petitions = pd.read_csv(f'{base_path}/petition.csv',
                        parse_dates=['start', 'end'])
petitions.shape

In [ ]:
petitions.describe()

In [ ]:
# 전체 데이터 중 투표가 500건 이상인 데이터를 기준으로 가져옵니다.
# 아웃라이어 데이터 제거를 위해 20만건 이상 데이터도 제거합니다.
petition_remove_outlier = petitions.loc[(petitions['votes'] > 500) &
                                        (petitions['votes'] < 200000)]
petition_remove_outlier.shape

In [ ]:
df = petition_remove_outlier.copy()

In [ ]:
df.describe()

# 이진분류 대상 정하기

In [ ]:
# 답변 대상 건
df.loc[df['answered'] == 1].shape

In [ ]:
df['votes'].plot.hist()

# 투표수를 평균보다 높게 혹은 적게 받는지 예측해 보기

In [ ]:
# 기본값을 0으로 설정
df['votes_pos_neg'] = 0

In [ ]:
# 평균 투표수
votes_mean = df['votes'].mean()
votes_mean

In [ ]:
# 투표수가 평균을 넘으면 1로
df['votes_pos_neg'] = (df['votes'] > votes_mean) == 1

In [ ]:
df['votes_pos_neg'].dtypes

In [ ]:
# 타입을 boolean 에서 int로 변경해 줍니다.
df['votes_pos_neg'] = df['votes_pos_neg'].astype(int)

In [ ]:
df[['votes', 'votes_pos_neg']].head()

In [ ]:
# 샘플로 보고 싶은 인덱스의 번호를 넣어주세요.
sample_index = 13

In [ ]:
sample_title = df.loc[sample_index, 'title']
sample_title

In [ ]:
sample_content = petitions.loc[sample_index, 'content']
sample_content

# 전처리 하기

In [ ]:
def preprocessing(text):
    # 개행문자 제거
    text = re.sub('\\\\n', ' ', text)
    # 특수문자 제거
    # 특수문자나 이모티콘 등은 때로는 의미를 갖기도 하지만 여기에서는 제거했습니다.
    # text = re.sub('[?.,;:|\)*~`’!^\-_+<>@\#$%&-=#}※]', '', text)
    # 한글, 영문, 숫자만 남기고 모두 제거하도록 합니다.
    # text = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z0-9]', ' ', text)
    # 한글, 영문만 남기고 모두 제거하도록 합니다.
    text = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z]', ' ', text)
    # 중복으로 생성된 공백값을 제거합니다.
    text = re.sub(' +', ' ', text)
    return text

In [ ]:
# 불용어 제거
def remove_stopwords(text):
    tokens = text.split(' ')
    stops = ['수', '현', '있는', '있습니다', '그', '년도', '합니다', '하는',
             '및', '제', '할', '하고', '더', '대한', '한', '그리고', '월',
             '저는', '없는', '입니다', '등', '일', '많은', '이런', '것은',
             '왜','같은', '같습니다', '없습니다', '위해', '한다']
    meaningful_words = [w for w in tokens if not w in stops]
    return ' '.join(meaningful_words)

In [ ]:
# 샘플데이터에 적용
%time pre_sample_content = preprocessing(sample_content)

In [ ]:
%time pre_sample_content = remove_stopwords(pre_sample_content)

In [ ]:
pre_sample_content

In [ ]:
%time df['content_preprocessing'] = df['content'].apply(preprocessing)

In [ ]:
%time df['content_preprocessed'] = df['content_preprocessing'].apply(remove_stopwords)

# 학습세트와 테스트세트 만들기
* 학습세트와 테스트세트를 7:3의 비율로 나눠 줍니다.

In [ ]:
df.head()

In [ ]:
df.shape

### 데이터를 나누는 기준을 구함

In [ ]:
split_count = int(df.shape[0] * 0.7)
split_count

### 학습세트 만들기

In [ ]:
df_train = df[:split_count].copy()
df_train.shape

In [ ]:
df_train.tail()

In [ ]:
# 학습 세트에서 투표수가 평균보다 많은 건
df_train.loc[df_train['votes_pos_neg'] == 1].shape

### 테스트세트 만들기
* split_count를 기준(아래에서 30% 갯수)으로 데이터를 사용

In [ ]:
df_test = df[split_count:].copy()
df_test.shape

In [ ]:
df_test.head()

In [ ]:
# 테스트 세트에서 투표수가 평균보다 많은 건
df_test.loc[df_test['votes_pos_neg'] == 1].shape

# 단어 벡터화 하기

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(analyzer = 'word',
                             tokenizer = None,
                             preprocessor = None,
                             stop_words = None,
                             min_df = 2,
                             ngram_range=(1, 3),
                             max_features = 2000
                            )
vectorizer

In [ ]:
%%time
train_feature_vector = vectorizer.fit_transform(df_train['content_preprocessed'])
train_feature_vector.shape

In [ ]:
%%time
test_feature_vector = vectorizer.transform(df_test['content_preprocessed'])
test_feature_vector.shape

In [ ]:
vocab = vectorizer.get_feature_names_out()
print(len(vocab))
vocab[:10]

In [ ]:
dist = np.sum(train_feature_vector, axis=0)

pd.DataFrame(dist, columns=vocab)

### TF-IDF 가중치 적용

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer
transformer = TfidfTransformer(smooth_idf=False)
transformer

In [ ]:
%%time
train_feature_tfidf = transformer.fit_transform(train_feature_vector)
train_feature_tfidf.shape

In [ ]:
%%time
test_feature_tfidf = transformer.transform(test_feature_vector)
test_feature_tfidf.shape

In [ ]:
test_feature_tfidf.shape

# LGBM으로 학습



In [ ]:
from lightgbm import LGBMClassifier

model = LGBMClassifier(random_state=42, n_jobs=1)
model

In [ ]:
# 학습에 사용할 y_label 을 넣어준다.
y_label = df_train['votes_pos_neg']
%time model = model.fit(train_feature_tfidf, y_label)

# 평가하기

In [ ]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

k_fold = KFold(n_splits=5, shuffle=True, random_state=42)

scoring = 'accuracy'
score = cross_val_score(model, train_feature_tfidf,
                        y_label, cv=k_fold, n_jobs=-1,
                        scoring=scoring)
score

In [ ]:
round(np.mean(score)*100,2)

# 예측

In [ ]:
y_pred = model.predict(test_feature_tfidf)
y_pred[:10]

In [ ]:
y_pred.shape

In [ ]:
# 예측 결과를 저장하기 위해 데이터프레임에 담아 준다.
output = pd.DataFrame(data={'votes_pos_neg_pred' : y_pred})
output.head()

In [ ]:
# 0과 1이 어떻게 집계 되었는지 확인한다.
# 실제 데이터에는 답변 대상 건이 있는데 없는 것으로 예측되었다.
output['votes_pos_neg_pred'].value_counts()

In [ ]:
df_test['votes_pos_neg_pred'] = y_pred

## 예측 결과의 정확도 보기

In [ ]:
df_test['pred_diff'] = np.abs(df_test['votes_pos_neg'] - df_test['votes_pos_neg_pred'])
df_test[['title', 'votes', 'votes_pos_neg', 'votes_pos_neg_pred', 'pred_diff']].head()

In [ ]:
pred_diff = df_test['pred_diff'].value_counts()
pred_diff

In [ ]:
print(f"전체 {y_pred.shape[0]}건의 데이터 중 {pred_diff[0]}건 예측")

In [ ]:
acc = ( pred_diff[0] / y_pred.shape[0] ) * 100
print(f'정확도 {acc:.6f}')